In [1]:
from accelerate import Accelerator
from transformers import AdamW, AutoTokenizer, AutoModelForCausalLM, get_scheduler
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm import tqdm

In [3]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

False

In [4]:
# Define the Tokenize function
def tokenize_function(examples):
    outputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    #outputs = tokenizer(examples["text"], return_tensors="pt", padding=True)
    return outputs

# Load the dataset and create a dataloader
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_dataset.set_format("torch")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


False

In [10]:
def training_function():
    
    model = AutoModelForCausalLM.from_pretrained("gpt2")
    
    # Initialize the accelerator
    accelerator = Accelerator()
    
    optimizer = AdamW(model.parameters(), lr=3e-5)

    dataloader = DataLoader(tokenized_dataset, batch_size=8, shuffle=True)

    # Move the model to the device
    dataloader, model, optimizer = accelerator.prepare(
        dataloader, model, optimizer
    )

    num_epochs = 3
    num_training_steps = num_epochs * len(dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    progress_bar = tqdm(range(num_training_steps))
    model.train()
    
    for epoch in range(num_epochs):
        for batch in dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
    
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)


In [11]:
from accelerate import notebook_launcher

notebook_launcher(training_function, num_processes=3)

Launching training on 3 GPUs.


/home/leczhang/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/leczhang/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/leczhang/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|                                   

RuntimeError: An issue was found when launching the training: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/home/leczhang/.local/lib/python3.10/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/home/leczhang/.local/lib/python3.10/site-packages/accelerate/utils/launch.py", line 543, in __call__
    self.launcher(*args)
  File "/tmp/ipykernel_3255929/1076316162.py", line 31, in training_function
    outputs = model(**batch)
  File "/home/leczhang/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/leczhang/.local/lib/python3.10/site-packages/torch/nn/parallel/distributed.py", line 1156, in forward
    output = self._run_ddp_forward(*inputs, **kwargs)
  File "/home/leczhang/.local/lib/python3.10/site-packages/torch/nn/parallel/distributed.py", line 1110, in _run_ddp_forward
    return module_to_run(*inputs[0